In [17]:
import pandas as pd
from spacy.lang.es import Spanish


In [81]:
# Load Dataset
data_raw = pd.read_csv('./train_sample.csv',index_col='index',dtype={'title':str,'label_quality':str,'language':str,'category':str})

In [61]:
# Basic statistics
data_raw.dtypes

title            object
label_quality    object
language         object
category         object
dtype: object

In [82]:
data_raw['title'] = data_raw.title.astype(str)
data_raw['category'] = data_raw.category.astype(str)

In [83]:
data_raw.describe()

,title,label_quality,language,category
count,1000000,1000000,1000000,1000000
unique,999975,2,2,1588
top,Boss Super Chorus,unreliable,portuguese,PANTS
freq,2,940691,500258,1867


In [84]:
data_raw.sample(5)

,title,label_quality,language,category
index,,,,
6061,Individuales Personalizados Infantiles Mantele...,unreliable,spanish,PLACEMATS
11696245,Auto Vehículo Reparación Herramientas Coches C...,unreliable,spanish,AIR_COMPRESSORS
4283164,Areia Granulado Higiênico Progato Pinus (madei...,unreliable,portuguese,CATS_LITTER
18090581,Pastillas De Freno Renault Master Trasera Fimpex,unreliable,spanish,VEHICLE_BRAKE_PADS
16940190,Resistência Corona Minha Ducha 6200w 220w 1 Peça,unreliable,portuguese,SHOWER_HEADS


In [85]:
data_raw.label_quality.value_counts()

unreliable    940691
reliable       59309
Name: label_quality, dtype: int64

In [87]:
data_raw.language.value_counts()

portuguese    500258
spanish       499742
Name: language, dtype: int64

In [88]:
data_raw.category.value_counts()

PANTS                                   1867
COFFEE_MAKERS                           1782
MUSICAL_KEYBOARDS                       1679
BABY_CAR_SEATS                          1671
MATTRESSES                              1660
                                        ... 
FORCE_GAUGES                               9
HONEY_EXTRACTORS                           8
COMMERCIAL_POPCORN_MACHINES                7
COLD_FOOD_AND_DRINK_VENDING_MACHINES       7
HAMBURGER_FORMERS                          3
Name: category, Length: 1588, dtype: int64

In [44]:
data_raw.dtypes

title            object
label_quality    object
language         object
category         object
dtype: object

In [18]:
#SPACY
nlp = Spanish()

In [19]:
nlp

In [89]:
test = list(data_raw[:50]['title'].values)

In [90]:
test

['Chaleco De Pesca Talle M C/manchas Ver Fotos. Cordoba',
 'Mocos Nasal Seguro Higinico Limpiador Deaspirador',
 'Mascara De Pestañas Extra Volumen 3 En 1 Ricosti',
 'Batidora De Mano Oster 2532 6 Velocidades 250 W Garantia ',
 'Kirkland Signature Pañales Supremo Tamaño 5, 150 Cantidad',
 'Balança Digital Upx 15 Kg',
 'Nexgard Pastilla Para Pulgas Y Garrapatas Hasta 25kg.  ',
 'Vara De Pesca Reel Combo Telesc\x9d\x9dpica P\x9d\x9dlo De Pesca De Fibr',
 'Serpentina Condensador Carrier 18000 Btuh',
 'Kit 10 Carimbos Premium 60 (só Carcaça)',
 'Braco Curvo Suspensao Traseira Esquerda Bmw 550 2011 **5626',
 'Pantalla Gigante Con Trípode  Loch 100 Pulg  2.00 X 1.50',
 'Colete Salva Vidas Xfloat Infantil Monster',
 'Navegador Gps Automotivo Mercedes C250 Tela 4.3 Touch Voz Tv',
 'Mochila Hidratante Camelback Incluye Bolsa De Agua 2.5 L ',
 'Cámara De Fotos Powershot Canon Sx40 En Excelente Estado',
 'Palmera Imperial, Real Cubana, Roistonea 2m - Granpalmera',
 'Parche Termoadhesivo Escudo De

In [67]:
my_doc = nlp(test)

# Create list of word tokens
token_list = []
for token in my_doc:
    token_list.append(token.text)
print(token_list)

['Chaleco', 'De', 'Pesca', 'Talle', 'M', 'C', '/', 'manchas', 'Ver', 'Fotos', '.', 'Cordoba']


In [68]:
for chunk in my_doc.noun_chunks:
   print(chunk.text, chunk.root.text, chunk.root.dep_,
          chunk.root.head.text)

ValueError: [E029] noun_chunks requires the dependency parse, which requires a statistical model to be installed and loaded. For more info, see the documentation:
https://spacy.io/usage/models